In [ ]:
import os
import json
from copy import copy

import pandas as pd
import numpy as np

from covidcaremap.data import external_data_path, processed_data_path

# Get total & ICU staffed bed counts for every acute hospital facility in USA

Following methodology from this paper: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5514420/

> Study Design and Data Sources

> We performed a repeated-measures time series analysis of US ICU bed supply during the 16-year period between 1996 and 2011. We obtained data on hospital characteristics and intensive care occupancy from the Centers for Medicare and Medicaid Services Hospital Cost Report Information System (HCRIS), a publicly available hospital-level database with detailed information on structural, organizational and cost data for all US hospitals. We excluded skilled nursing facilities, long term acute care hospitals, hospitals located in US territories and stand-alone pediatric hospitals (1, 2). We augmented the HCRIS data with data from the US Census Bureau’s 2010 urban-rural classification file which we used to designate hospitals as urban or rural by ZIP code (5).

> Variables

> The primary dependent variable was each hospital’s number of ICU beds compared to the previous year. We defined total ICU beds using the summed counts of four HCRIS bed categories that were available throughout the study interval: intensive care beds, surgical intensive care beds, cardiac intensive care beds and burn intensive care beds 

## Useful References & Links:

CMS Healthcare Cost Report Information System (HCRIS):
https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/Cost-Reports/Hospital-2010-form

Hospital Facilities: 
- http://downloads.cms.gov/files/hcris/hosp10-reports.zip

Public Use File (annual, from 2015): 
- https://data.cms.gov/api/views/absp-nd3x/files/e0ca9126-8fd6-42ca-82bf-c2fe40bd4c0e?download=true&filename=CostReport_Documentation_2015_Final_Oct2019.xlsx
- https://www.cms.gov/files/zip/hospital-cost-report-public-use-file-2015.zip

![](https://www.resdac.org/sites/resdac.umn.edu/files/kb-images/Figure%204_2.png)

Direct link to 2018 reporting from hospitals: http://downloads.cms.gov/Files/hcris/HOSP10FY2018.zip


## Specific guidance for what we're doing here:
from CMS data research guide: https://www.resdac.org/articles/medicare-cost-report-data-structure

![alt text](https://www.resdac.org/sites/resdac.umn.edu/files/kb-images/Figure%205.PNG)

> In summary, the number of beds will be located in the numeric file. 

> To identify the number of beds for every report in numeric file, filter the records where the second column (Worksheet Indicator) is “S300001,” the third column (Line Number) is “01400”, and the fourth column (Column Number) is “00200”. 

> To identify the number of beds for a specific report submitted by a specific facility, filter the records by the “Record Report Number,” which is reported in Column 1. The Report Record Number for a specific facility can be found in the Report data file.

## Official Worksheet Definitions

from Provider Reimbursement Manual (https://www.cms.gov/Regulations-and-Guidance/Guidance/Manuals/Paper-Based-Manuals-Items/CMS021935):

Column 2--Refer to 42 CFR 412.105(b) and 69 FR 49093-49098 (August 11, 2004) to determine the facility bed count. Indicate the number of beds available for use by patients at the end of the cost reporting period.

A bed means an adult bed, pediatric bed, portion of inpatient labor/delivery/postpartum (LDP) room (also referred to as birthing room) bed when used for services other than labor and delivery, or newborn ICU bed (excluding newborn bassinets) maintained in a patient care area for lodging patients in acute, long term, or domiciliary areas of the hospital. Beds in post-anesthesia, post- operative recovery rooms, outpatient areas, emergency rooms, ancillary departments (however, see exception for labor and delivery department), nurses' and other staff residences, and other such areas that are regularly maintained and utilized for only a portion of the stay of patients (primarily for special procedures or not for inpatient lodging) are not termed a bed for these purposes. (See CMS Pub. 15-1, chapter 22, §2205.)

For cost reporting periods beginning prior to October 1, 2012, beds in distinct ancillary labor and delivery rooms and the proportion of LDP room (birthing room) beds used for labor and delivery services are not a bed for these purposes. (See 68 FR 45420 (August 1, 2003).)

For cost reporting periods beginning on or after October 1, 2012, in accordance with 77 FR 53411- 53413 (August 31, 2012), beds in distinct labor and delivery rooms, when occupied by an inpatient receiving IPPS-level acute care hospital services or when unoccupied, are considered to be part of a hospital’s inpatient available bed count in accordance with 42 CFR 412.105(b) and are to be reported on line 32. Furthermore, the proportion of the inpatient LDP room (birthing room) beds used for ancillary labor and delivery services is considered part of the hospital’s available bed count.

Column 8--Enter the number of inpatient days for all classes of patients for each component. Include organ acquisition and HMO days in this column. This amount will not equal the sum of columns 5 through 7, when the provider renders services to other than titles V, XVIII, or XIX patients.

Line 1--For cost reporting periods beginning before October 1, 2012, exclude from column 2 the portion of LDP room (birthing room) beds used for ancillary labor and delivery services, but include on this line beds used for routine adult and pediatric services (postpartum). In accordance with the instructions in 68 FR 45420 (August 1, 2003), compute this proportion (off the cost report) by multiplying the total number of occupied and unoccupied available beds in the LDP room by the percentage of time these beds were used for ancillary labor and delivery services. An example of how to calculate the “percentage of time” would be for a hospital to determine the number of hours for the cost reporting period during which each LDP room maternity patient received labor and delivery services and divide the sum of those hours for all such patients by the sum of the total hours (for both, ancillary labor and delivery services and for routine postpartum services) that all maternity patients spent in the LDP room during that cost reporting period. Alternatively, a hospital could calculate an average percentage of time maternity patients received ancillary labor and delivery services in an LDP room during a typical month.
For cost reporting periods beginning on or after October 1, 2012, include all the available LDP room (birthing room) beds in the available bed count in column 2. (See 77 FR 53411-53413 (August 31, 2012).) The proportion of available LDP room beds related to the ancillary labor and delivery services must not be excluded from column 2 for those cost reporting periods.
In columns 5, 6, 7 and 8, enter the number of adult and pediatric hospital days excluding the SNF and NF swing-bed, observation bed, and hospice days. In columns 6 and 7, also exclude HMO days. Do not include in column 6 Medicare Secondary Payer/Lesser of Reasonable Cost (MSP/LCC) days. Include these days only in column 8. However, do not include employee discount days in column 8.

Line 7--Enter the sum of lines 1, 5, and 6.
Lines 8 through 13--Enter the appropriate statistic applicable to each discipline for all programs.
Line 14--Enter the sum of lines 7 through 13 for columns 2 through 8, and for columns 12 through 15, enter the amount from line 1. For columns 9 through 11, enter the total for each from your records. Labor and delivery days (as defined in the instructions for Worksheet S-3, Part I, line 32) must not be included on this line.




## Methods

This notebook gathers the HCRIS information based on the following inputs:

- The hospital data at http://downloads.cms.gov/files/hcris/hosp10-reports.zip 
  - HOSPITAL10_PROVIDER_ID_INFO.CSV (`HOSPITAL10`): Provides facility level IDs, names and addresses
- The 2018 reporting data from http://downloads.cms.gov/Files/hcris/HOSP10FY2018.zip
  - hosp10_2018_RPT.CSV (`HOSP10_RPT`): Provides facility report information.
  - hosp10_2018_NMRC.CSV (`HOSP10_NMRC`): Contains the numeric column values that are linked back to the report data from above.
- A data dictionary at https://www.cms.gov/files/zip/hospital2010-documentation.zip
  - HCRIS_DataDictionary.csv: provides report column codes with titles.

It then takes the following steps:

- Join the `HOSPITAL10` data and `HOSP10_RPT` to get all information about a facility per report.
- Filter the `HOSP10_NMRC` to only those line numbers and columns we care about
  - Filter `Line Number` to keep information about Staffed Beds, Staffed Bed Days, and Inpatient Days AND
  - Filter `Column Number` to keep information concerning the numeric counts related to: 
    - Hospital Adult and Peds
    - Intensive Care Unit 
    - Coronary Care Unit
    - Burn ICU
    - Surgical ICU 
    - Total
- Join the filtered `HOSP10_NMRC` data to the previously joined `HOSPITAL10` and `HOSP10_RPT` data.
- Aggregate counts for 'Intensive Care Unit', 'Coronary Care Unit', 'Burn ICU', 'Surgical ICU' into 'ICU Total Staffed Beds',  'ICU Total Bed Days Available', 'ICU Total Inpatient Days'.
- Calculate 'ICU Occupancy Rate' and 'Total Bed Occupancy Rate' using the ratio of Inpatient Days to Bed Day Available for ICU and over all totals. If the '* Bed Days' columns are not available, then compute them by using the number of staffed bed multiplied by the number of days between the report's Fiscal Year Begin Date and Fiscal Year End Date
- The resulting dataset will have information per Record per Facility. Some facility have multiple records. Choose the record with the 'Fiscal Year End Date' being the most recent.
- Drop problematic facilities, e.g. PARKVIEW MEDICAL CENTER which reports 2,290,239,239 total beds. 
- Join with the `usa_hospital_beds_hcris2018_geocoded.geojson` dataset based on 'Provider Number', generate a new GeoJSON file that contains all aggregated counts and other relevant HCRIS information with this joined data.


### Load the HCRIS Facility data

In [ ]:
hosp_df = pd.read_csv(external_data_path('HCRIS-HOSPITAL10_PROVIDER_ID_INFO.CSV'))
hosp_df.head()

In [ ]:
# provider num should be 6 char so need to zfill
hosp_df['PROVIDER_NUMBER'] = hosp_df['PROVIDER_NUMBER'].apply(lambda x: str(x).zfill(6))

# Rename this column to match up with reports
hosp_df = hosp_df.rename(columns={'PROVIDER_NUMBER': 'Provider Number'})

In [ ]:
# Show all providers in San Francisco county
hosp_df[hosp_df['County'] == 'SAN FRANCISCO']

### Use the HCRIS data dictionary to generate data mappings

In [ ]:
hcris_dict = pd.read_csv(external_data_path('HCRIS-HCRIS_DataDictionary.csv'))
hcris_dict.head()

In [ ]:
data_dict = {c:t for c,t in zip(hcris_dict['Column Code'],hcris_dict['Title'])}
data_dict

In [ ]:
# Report Table file columns
rpt_columns = [
               'RPT_REC_NUM',
               'PRVDR_CTRL_TYPE_CD',
               'PRVDR_NUM',
               'NPI',
               'RPT_STUS_CD',
               'FY_BGN_DT',
               'FY_END_DT',
               'PROC_DT',
               'INITL_RPT_SW',
               'LAST_RPT_SW',
               'TRNSMTL_NUM',
               'FI_NUM',
               'ADR_VNDR_CD',
               'FI_CREAT_DT',
               'UTIL_CD',
               'NPR_DT',
               'SPEC_IND',
               'FI_RCPT_DT'
]
[data_dict[col] for col in rpt_columns]

In [ ]:
# Numerical Table file columns
nmrc_columns = [
             'RPT_REC_NUM',
             'WKSHT_CD',
             'LINE_NUM',
             'CLMN_NUM',
             'ITM_VAL_NUM'
]
[data_dict[col] for col in nmrc_columns]

In [ ]:
# Maps to 'Line Number' in the numeric report
beds_dict = {
    'Hospital Adult and Peds': '00100',
    'Intensive Care Unit': '00800',
    'Coronary Care Unit': '00900',
    'Burn ICU': '01000',
    'Surgical ICU': '01100',
    'Total': '01400'
}

# Maps to 'Column Number' in numeric report
value_count_dict = {
    'Staffed Beds': '00200',
    'Bed Days Available': '00300',
    'Inpatient Days': '00800'
}

icu_beds = ['Intensive Care Unit', 'Coronary Care Unit', 'Burn ICU', 'Surgical ICU']
icu_staffed_beds_columns = ['{} Staffed Beds'.format(x) for x in icu_beds]
icu_bed_days_columns = ['{} Bed Days Available'.format(x) for x in icu_beds]
icu_inpatient_days_columns = ['{} Inpatient Days'.format(x) for x in icu_beds]

all_count_columns = [
    'ICU Total Staffed Beds', 
    'ICU Total Bed Days Available',
    'ICU Total Inpatient Days',
    'ICU Occupancy Rate',
    'Total Bed Occupancy Rate'
]

for bed_desc in beds_dict:
    for value_desc in value_count_dict:
        column_name = '{} {}'.format(bed_desc, value_desc)
        all_count_columns.append(column_name)

In [ ]:
beds_dict_flip = {v:k for k,v in beds_dict.items()}
bedtype_list = list(beds_dict_flip.keys())

value_count_dict_flip = {v:k for k,v in value_count_dict.items()}
value_count_list = list(value_count_dict_flip.keys())

### Load the HCRIS report file

In [ ]:
hosp10_rpt_df = pd.read_csv(external_data_path('HCRIS-hosp10_2018_RPT.CSV'), 
                            names=[data_dict[col] for col in rpt_columns], 
                            dtype={'Provider Number':object})
hosp10_nmrc_df = pd.read_csv(external_data_path('HCRIS-hosp10_2018_NMRC.CSV'),  
                             names=[data_dict[col] for col in nmrc_columns], 
                             dtype={'Line Number':object, 'Column Number':object})    

In [ ]:
hosp10_rpt_df[hosp10_rpt_df['Provider Number'] == '010032']

In [ ]:
hosp_df[hosp_df['Provider Number'] == '010032']

In [ ]:
hosp10_nmrc_df[hosp10_nmrc_df['Report Record Number'] == 623132] 

### Join the data and format into counts

#### Filter hosp10_2018_nmrc_df records to just what we care about and index by report number.

In [ ]:
filter_condition = (hosp10_nmrc_df['Worksheet Identifier'] == 'S300001') & \
                   (hosp10_nmrc_df['Column Number'].isin(value_count_list)) & \
                   (hosp10_nmrc_df['Line Number'].isin(bedtype_list))
filtered_record_df = hosp10_nmrc_df[filter_condition].set_index('Report Record Number')
filtered_record_df.count()


#### Join filtered numeric records, HOSP10_RPT, and HOSPITAL10 data

In [ ]:
hosp_and_rpt = hosp10_rpt_df.join(hosp_df.set_index('Provider Number'), 
                                 on='Provider Number')
hosp_and_rpt_and_records = hosp_and_rpt.join(filtered_record_df, on='Report Record Number')


In [ ]:
hosp_and_rpt_and_records[['Provider Number', 'Process Date', 'Report Record Number', 'Line Number', 'Column Number', 'Item Value Number']].head()

#### Group the data, creating columns that contain the count values we are interested in

This step creates columns that have a 0 value for all rows in the dataframe except those that match the Line Number and Column Number for the target counts; for these rows, the value will be the count value in 'Item Number Value'.
We then perform a "groupby" operation that will sum lines up per Report Record Number and Provider Number, so that we end up with a row per report that has all the counts of interest.

In [ ]:
report_cells_to_be_grouped = hosp_and_rpt_and_records.copy()

In [ ]:
count_columns = []
for bed_desc, bed_key in beds_dict.items():
    for value_desc, value_key in value_count_dict.items():
        column_name = '{} {}'.format(bed_desc, value_desc)
        count_columns.append(column_name)
        def column_mapper(row):            
            if row['Line Number'] == bed_key and row['Column Number'] == value_key:
                return row['Item Value Number']
            else:
                return 0
        report_cells_to_be_grouped[column_name] = report_cells_to_be_grouped.apply(column_mapper, axis=1)

In [ ]:
report_cells_to_be_grouped

In [ ]:
record_sum_columns = ['Provider Number', 'Report Record Number'] + count_columns
provider_record_sums = report_cells_to_be_grouped[record_sum_columns].groupby(
    ['Provider Number', 'Report Record Number']).sum()

# Merge in report dates to use in case bed days available is missing
provider_record_sums = provider_record_sums.reset_index().merge(
    report_cells_to_be_grouped[['Report Record Number', 
                                'Fiscal Year Begin Date',
                                'Fiscal Year End Date']], on='Report Record Number'
)

### Calculate ICU totals and Occupancy rates

In the case where Bed Days are 0, they are computed using the staffed bed counts and the duration of the reporting periods.

In [ ]:
provider_record_sums['ICU Total Staffed Beds'] = provider_record_sums[icu_staffed_beds_columns].sum(axis=1)
provider_record_sums['ICU Total Bed Days Available'] = provider_record_sums[icu_bed_days_columns].sum(axis=1)
provider_record_sums['ICU Total Inpatient Days'] = provider_record_sums[icu_inpatient_days_columns].sum(axis=1)

provider_record_sums['beg_date'] = pd.to_datetime(provider_record_sums['Fiscal Year Begin Date'])
provider_record_sums['end_date'] = pd.to_datetime(provider_record_sums['Fiscal Year End Date'])
provider_record_sums['days'] = (provider_record_sums['end_date'] -
                                provider_record_sums['beg_date']).dt.days

provider_record_sums.loc[provider_record_sums['ICU Total Bed Days Available'] == 0,
                         'ICU Total Bed Days Available'] = \
    provider_record_sums['days'] * provider_record_sums['ICU Total Staffed Beds']
provider_record_sums.loc[provider_record_sums['Total Bed Days Available'] == 0,
                         'Total Bed Days Available'] = \
    provider_record_sums['days'] * provider_record_sums['Total Staffed Beds']

provider_record_sums = provider_record_sums.drop(columns=['Fiscal Year Begin Date',
                                       'Fiscal Year End Date',
                                       'beg_date',
                                       'end_date',
                                       'days'])

provider_record_sums['ICU Occupancy Rate'] = provider_record_sums['ICU Total Inpatient Days']/provider_record_sums['ICU Total Bed Days Available']
provider_record_sums.loc[provider_record_sums['ICU Total Bed Days Available'] == 0, 
                         'ICU Occupancy Rate'] = 0.0

provider_record_sums['Total Bed Occupancy Rate'] = provider_record_sums['Total Inpatient Days']/provider_record_sums['Total Bed Days Available']
provider_record_sums.loc[provider_record_sums['Total Bed Days Available'] == 0, 
                         'Total Bed Occupancy Rate'] = 0.0


### Join with full report data

In [ ]:
provider_record_sums = provider_record_sums.set_index(['Provider Number', 'Report Record Number'])
full_df = hosp_and_rpt.set_index(['Provider Number', 'Report Record Number']).join(provider_record_sums)


### Filter to a single report by dropping all reports besides one with latest fiscal year end date

In [ ]:
full_df = full_df.reset_index()
full_df['Fiscal Year End Date'] = pd.to_datetime(full_df['Fiscal Year End Date'])
full_df = full_df.sort_values('Fiscal Year End Date', ascending=False)

In [ ]:
# check that duplicates are dropped by Fiscal Year End Date correctly
dup_provider_nums = full_df[full_df.duplicated('Provider Number')]['Provider Number'].values
full_df.loc[full_df['Provider Number'].isin(dup_provider_nums)].sort_values('Provider Number')

In [ ]:
full_df = full_df.drop_duplicates('Provider Number')

In [ ]:
# check that duplicates are dropped by Fiscal Year End Date correctly
full_df.loc[full_df['Provider Number'].isin(dup_provider_nums)].sort_values('Provider Number')

### Generate final dataframe of provider information

In [ ]:
final_columns = list(hosp_df.columns.values) + list(provider_record_sums.columns.values)

In [ ]:
final_df = full_df[final_columns]

In [ ]:
final_df.head()

### Drop bad facility data

In [ ]:
# Drop PARKVIEW MEDICAL CENTER which reports over 2 million staffed beds.
final_df = final_df[final_df['Provider Number'] != '060020']

### Match data to geocoded facility information

These points are geocoded from the origina v1 notebook `usa_hcris2018_facilitybedcounts_20200313_v1.ipynb`.

Notes from original GeoCoding:

- geocoding first with google maps which gave best results, fall back to mapbox geocode and then search_str without street address
- TODO: figure out why geocoder with gmaps stopped sending requests halfway through, temp switch to directly using gmaps api

In [ ]:
geojson = json.loads(open(processed_data_path('usa_facilities_hcris_geocoded.geojson')).read())

In [ ]:
## Key our data by Provider Number
final_feature_data = final_df.to_dict(orient='records')
final_feature_data_keyed = dict((r['Provider Number'], r) for r in final_feature_data)

## Use the HOSPITAL10 data to account for facilities that don't have report data
hosp_feature_data = hosp_df.to_dict(orient='records')
hosp_feature_data_keyed = dict((r['Provider Number'], r) for r in hosp_feature_data)

#### Create empty facility information for facilities without HCRIS records

In [ ]:
nodata_properties = copy(final_feature_data[0])
for key in nodata_properties:
    if type(nodata_properties[key]) is int:
        nodata_properties[key] = 0
    elif type(nodata_properties[key]) is float:
        nodata_properties[key] = 0.0
    else:
        nodata_properties[key] = 'NoData'

In [ ]:
def generate_properties(provider_number):
    if provider_number in final_feature_data_keyed:
        new_properties = final_feature_data_keyed[provider_number]
        return new_properties, True
    else:
        new_properties = copy(nodata_properties)
        new_properties['Provider Number'] = provider_number
        if provider_number in hosp_feature_data_keyed:
            hosp_info = hosp_feature_data_keyed[provider_number]
            for key in new_properties:
                if key in hosp_info:
                    new_properties[key] = hosp_info[key]
        return new_properties, False

#### Replace properties from geocoded facilities with calculated HRCIS properties

In [ ]:
facilities_with_data = 0
facilities_without_data = 0
for feature in geojson['features']:
    provider_number = feature['properties']['Provider Number']
    new_props, found = generate_properties(provider_number)
    if found:
        facilities_with_data += 1
    else:
        facilities_without_data += 1
        
    feature['properties'] = new_props
    
print('{} Found, {} not found'.format(facilities_with_data, facilities_without_data))

In [ ]:
geojson['features'][0]

### Write usa_hospital_beds_hcris2018 GeoJSON

In [ ]:
with open(processed_data_path('usa_hospital_beds_hcris2018.geojson'), 'w') as f:
    f.write(json.dumps(geojson, indent=4))